In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import imblearn
import collections

import sklearn
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report,confusion_matrix

from keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

from collections import Counter
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN

import pickle

Load the Data

In [2]:
csv_file = 'actionstatespaceTMomoUBharat_combined.csv'
dataframe = pd.read_csv(csv_file)

In [3]:
dataframe.head()

#print(dataframe["Time Since Last Charge"].mean())
#print(dataframe["Time Since Last Action Formatted"].mean())

,Action,Last Action,Session Time,Time Since Last Charge,Session Time Formatted,Time Since Last Action,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,...,st1,st2,st3,st4,st5,st6,st7,st8,st9,st10
0,2,17,0:01:23.171358,0.00,1.38,0:00:01.000000,0.02,Sad,1,Low,...,0,0,01:23.171358,01:23,1,1,1,23,0.38,1.38
1,10,2,0:01:36.964613,0.22,1.60,0:00:13.793255,0.22,Sad,1,Low,...,0,0,01:36.964613,01:36,1,1,1,36,0.60,1.60
2,9,10,0:01:59.253713,0.60,1.98,0:00:22.289100,0.37,Sad,1,Low,...,0,0,01:59.253713,01:59,1,1,1,59,0.98,1.98
3,4,9,0:02:17.053963,0.90,2.28,0:00:17.800250,0.28,Sad,1,Low,...,0,0,02:17.053963,02:17,2,2,2,17,0.28,2.28
4,8,4,0:02:38.159474,1.25,2.63,0:00:21.105511,0.35,Sad,1,None,...,0,0,02:38.159474,02:38,2,2,2,38,0.63,2.63


Format and One-hot encode the data

In [4]:
dataframe.drop("tsla1", axis=1, inplace=True)
dataframe.drop("tsla2", axis=1, inplace=True)
dataframe.drop("tsla3", axis=1, inplace=True)
dataframe.drop("tsla4", axis=1, inplace=True)
dataframe.drop("tsla5", axis=1, inplace=True)
dataframe.drop("tsla6", axis=1, inplace=True)
dataframe.drop("st1", axis=1, inplace=True)
dataframe.drop("st2", axis=1, inplace=True)
dataframe.drop("st3", axis=1, inplace=True)
dataframe.drop("st4", axis=1, inplace=True)
dataframe.drop("st5", axis=1, inplace=True)
dataframe.drop("st6", axis=1, inplace=True)
dataframe.drop("st7", axis=1, inplace=True)
dataframe.drop("st8", axis=1, inplace=True)
dataframe.drop("st9", axis=1, inplace=True)
dataframe.drop("st10", axis=1, inplace=True)
dataframe.drop("Session Time", axis=1, inplace=True)
dataframe.drop("Time Since Last Action", axis=1, inplace=True)

dataframe.drop("Session Time Formatted", axis=1, inplace=True)
#dataframe.drop("Time Since Last Charge", axis=1, inplace=True)  

In [5]:
dataframe.head(100)

,Action,Last Action,Time Since Last Charge,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,Input Mood,Activity
0,2,17,0.00,0.02,Sad,1,Low,Neutral,Studying
1,10,2,0.22,0.22,Sad,1,Low,Neutral,Studying
2,9,10,0.60,0.37,Sad,1,Low,Neutral,Studying
3,4,9,0.90,0.28,Sad,1,Low,Neutral,Studying
4,8,4,1.25,0.35,Sad,1,None,Neutral,Studying
...,...,...,...,...,...,...,...,...,...
95,14,11,17.24,1.38,Happy,1,Low,Neutral,Studying
96,11,14,19.49,2.25,Sad,1,Low,Neutral,Studying
97,1,11,20.09,0.58,Sad,1,Low,Neutral,Studying
98,10,1,21.70,1.60,Sad,1,None,Neutral,Studying


In [6]:
pd.set_option('display.max_columns', None)
dataframe = pd.get_dummies(dataframe, columns=["Activity"], prefix=["Activity"])
dataframe = pd.get_dummies(dataframe, columns=["Camera Mood"], prefix=["Camera_Mood"])
dataframe = pd.get_dummies(dataframe, columns=["Input Mood"], prefix=["Input Mood"])
#both = pd.concat([dataframe, dum_df])
#dataframe.drop("Input Mood", axis=1, inplace=True) 
#both.head(5)

dataframe = pd.get_dummies(dataframe, columns=["Noise"], prefix=["Noise"])
dataframe = pd.get_dummies(dataframe, columns=["Last Action"], prefix=["Last Action"])
dataframe.head(100)

,Action,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
0,2,0.00,0.02,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,10,0.22,0.22,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,9,0.60,0.37,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,0.90,0.28,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,8,1.25,0.35,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,14,17.24,1.38,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
96,11,19.49,2.25,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
97,1,20.09,0.58,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
98,10,21.70,1.60,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
X = dataframe.drop('Action', axis=1)
y = dataframe['Action']

In [8]:
X.head(5)

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
0,0.00,0.02,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.22,0.22,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.60,0.37,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0.90,0.28,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1.25,0.35,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train = X[76:239]
X_test = X[0:76]
y_train = y[76:239]
y_test = y[0:76]

In [10]:
X_test

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
0,0.00,0.02,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.22,0.22,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.60,0.37,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0.90,0.28,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1.25,0.35,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,24.70,1.68,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
72,25.40,0.68,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
73,26.80,1.40,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
74,27.85,1.05,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [11]:
counter = Counter(y_train)
print(counter)

Counter({7: 18, 3: 16, 10: 16, 11: 16, 9: 15, 6: 13, 12: 11, 14: 11, 4: 10, 5: 9, 8: 9, 2: 8, 1: 5, 17: 3})


In [12]:
counter = Counter(y_test)
print(counter)

Counter({10: 9, 4: 9, 9: 8, 11: 8, 12: 6, 7: 6, 8: 5, 14: 5, 1: 4, 2: 3, 5: 3, 3: 3, 16: 3, 6: 2, 13: 1, 17: 1})


Normalize data (train and test done seperately)

In [13]:
X_train_scaled = X_train.copy()

col_names = ['Time Since Last Charge', 'Time Since Last Action Formatted']
features = X_train_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_train_scaled[col_names] = features

filename = 'TMomoUBharat_scaler2.sav'
#pickle.dump(scaler, open(filename, 'wb'))

In [14]:
scaler2 = pickle.load(open(filename, 'rb'))

X_test_scaled = X_test.copy()

features = X_test_scaled[col_names]
features = scaler2.transform(features.values)
X_test_scaled[col_names] = features

X_test_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
0,-1.601100,-1.706217,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-1.586164,-1.287449,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.560365,-0.973373,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,-1.539998,-1.161819,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,-1.516236,-1.015250,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


Load and Train the Model

In [15]:
mlp = MLPClassifier(max_iter=10000, activation='relu', hidden_layer_sizes=(15)) #doesn't converge at 500 so put to higher, do we want it to converge?
mlp

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [16]:
mlp.fit(X_train_scaled,y_train)

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [17]:
#filename = 'TMomoUBharat_model2.sav'
#pickle.dump(mlp, open(filename, 'wb'))

In [18]:
pred = mlp.predict(X_test_scaled)
pred

array([ 7, 14, 12, 12,  2,  3, 12, 10, 12, 10, 14,  6, 14,  9, 12, 14, 10,
        7,  3, 12, 14,  4, 10,  4, 10, 10, 10, 10,  9,  3,  9, 10,  9, 10,
       10,  9, 14, 10,  5, 10,  5, 10,  9, 10, 10, 14,  9, 10, 14,  9,  5,
        9, 10, 10, 14,  9, 11, 11,  6, 11, 14,  4, 10, 11,  9, 11,  9, 10,
       10, 10, 11, 10, 10,  9,  9, 10])

In [19]:
y_test.value_counts()

4     9
10    9
9     8
11    8
7     6
12    6
8     5
14    5
1     4
2     3
3     3
5     3
16    3
6     2
13    1
17    1
Name: Action, dtype: int64

In [20]:
from collections import Counter
Counter(pred)


Counter({7: 2,
         14: 10,
         12: 6,
         2: 1,
         3: 3,
         10: 26,
         6: 2,
         9: 14,
         4: 3,
         5: 3,
         11: 6})

In [58]:
y_test.to_csv('actual.csv')

In [107]:
prediction = pd.DataFrame(pred, columns=['predictions']).to_csv('prediction.csv')